In [2]:
from collections import defaultdict
import os 
import csv

def readFile(fn):
    f = open(fn,"r")
    for line in f:
        line = line.strip().rstrip(",")
        record = frozenset(line.split(','))
        yield record

def returnItemSetTL(data_iter):
    _itemSet = set()
    TransactionList = []
    for record in data_iter:
        trans = frozenset(record)
        TransactionList.append(trans)
        for item in trans:
            _itemSet.add(frozenset([item]))
    return _itemSet,TransactionList

def returnItemWithMinSupport(itemSet,TransactionList,ms,freqSet):
    localSet = defaultdict(int)
    _itemSet = set()
    for item in itemSet:
        for trans in TransactionList:
            if item.issubset(trans):
                localSet[item]+=1
                freqSet[item]+=1
    for item,count in localSet.items():
        support = float(count)/float(len(TransactionList))
        if support>=minSupport:
            _itemSet.add(item)
    return _itemSet

def joinSet(itemSet,length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j))==length])

def getSupport(freqSet,item,TransactionList):
    return float(freqSet[item])/float(len(TransactionList))

def runApriori(data_iter,ms,mc):
    itemSet,TransactionList = returnItemSetTL(data_iter)

    freqSet = defaultdict(int)
    largeSet = {}
    oneCSet = returnItemWithMinSupport(itemSet,TransactionList,ms,freqSet)
    k=2
    currentLSet = oneCSet
    print ("L1")
    print (currentLSet)
    while(currentLSet!=set([])):
        largeSet[k-1]=currentLSet
        currentLSet = joinSet(currentLSet,k)
        print ("L",k)
        print (currentLSet)
        currentCSet = returnItemWithMinSupport(currentLSet,TransactionList,ms,freqSet)
        currentLSet = currentCSet
        k+=1
    items = []
    for key,value in largeSet.items():
        items.extend([(tuple(item),getSupport(freqSet,item,TransactionList)) for item in value])
        print (items)
        max_value = 0
        for key,value in largeSet.items():
            items.extend([(tuple(item),getSupport(freqSet,item,TransactionList)) for item in value])
            print (items)
            max_value = 0
            for item,support in sorted(items,key= lambda support:support[1]):
                if max_value<len(item):
                    max_value=len(item)
                for item,support in sorted(items,key=lambda support:support[1]):
                    if max_value==len(item):
                        print (item)


if __name__=='__main__':
    file_location = os.path.join(os.path.expanduser('../'),'dataset','data.csv')
    inFile = readFile(file_location)
    #inFile = readFile("data.csv")
    minSupport = 0.6
    minConfidence = 1
    runApriori(inFile,minSupport,minConfidence)

L1
{frozenset({'Keychain'}), frozenset({'Mango'}), frozenset({'Onion'}), frozenset({'Chocolate'}), frozenset({'Eggs'})}
L 2
{frozenset({'Eggs', 'Keychain'}), frozenset({'Eggs', 'Mango'}), frozenset({'Keychain', 'Mango'}), frozenset({'Eggs', 'Chocolate'}), frozenset({'Chocolate', 'Mango'}), frozenset({'Eggs', 'Onion'}), frozenset({'Onion', 'Mango'}), frozenset({'Chocolate', 'Onion'}), frozenset({'Chocolate', 'Keychain'}), frozenset({'Onion', 'Keychain'})}
L 3
{frozenset({'Eggs', 'Onion', 'Keychain'}), frozenset({'Onion', 'Keychain', 'Mango'}), frozenset({'Chocolate', 'Onion', 'Keychain'}), frozenset({'Eggs', 'Keychain', 'Mango'}), frozenset({'Eggs', 'Chocolate', 'Keychain'}), frozenset({'Chocolate', 'Keychain', 'Mango'})}
L 4
set()
[(('Keychain',), 1.0), (('Mango',), 0.6), (('Onion',), 0.6), (('Chocolate',), 0.6), (('Eggs',), 0.8)]
[(('Keychain',), 1.0), (('Mango',), 0.6), (('Onion',), 0.6), (('Chocolate',), 0.6), (('Eggs',), 0.8), (('Keychain',), 1.0), (('Mango',), 0.6), (('Onion',), 0